In [1]:
from gurobipy import Model, quicksum, GRB

In [2]:
Cust = set(range(1,3))
DC = Cust.copy() 
Route = {(i,j) for i in DC for j in Cust}
CT = {1,2}

T = 4
Period = list(range(1, T+1))

In [3]:
transport = {(i,t):10 for i in DC for t in Period}
transport.update({(i,1):1 for i in DC})
delivery = {(i,j):5 for i,j in Route}
dc_run = {(i,ct):100*ct for i in DC for ct in CT}
dc_stock = {i:1 for i in DC}
dc_ub = {ct:40*ct for ct in CT}

demand = {(j,t):10 for j in Cust for t in Period}

In [4]:
model = Model()


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only


In [5]:
X, x, y, sD = {}, {}, {}, {}

for i in DC:
    for t in Period:
        X[i,t] = model.addVar(vtype="C", name=f'X[{i},{t}]')

for i,j in Route:
    for t in Period:
        x[i,j,t] = model.addVar(vtype="C", name=f'x[{i},{j},{t}]')

for i in DC:
    for ct in CT:
        for t in Period:
            y[i,ct,t] = model.addVar(vtype="B", name=f'y[{i},{ct},{t}]')

for i in DC:
    sD[i,0] = model.addVar(vtype="C", ub=0, name=f'sD[{i},{t}]')
    for t in Period:
        sD[i,t] = model.addVar(vtype="C", name=f'sD[{i},{t}]')

model.update()

In [6]:
Cust_Demand_Cons, DC_Running_Cons, CT_Only_Cons, DC_Flow_Cons, DC_UB_Cons = {}, {}, {}, {}, {}

for j in Cust:
    for t in Period:
        Cust_Demand_Cons[j,t] = model.addConstr(
            quicksum(x[i,j,t] for i in DC)
            ==
            demand[j,t]
        )

for i,j in Route:
    for t in Period:
        DC_Running_Cons[i,j] = model.addConstr(
            x[i,j,t]
            <=
            quicksum(demand[j,t_] for t_ in Period) * quicksum(y[i,ct,t] for ct in CT)
        )

for i in DC:
    for t in Period:
        CT_Only_Cons[i,t] = model.addConstr(
            quicksum(y[i,ct,t] for ct in CT)
            <=
            1
        )

for i in DC:
    for t in Period:
        DC_Flow_Cons = model.addConstr(
            X[i,t] + sD[i,t-1]
            ==
            quicksum(x[i,j,t] for j in Cust) + sD[i,t]
        )

for i in DC:
    for t in Period:
        DC_UB_Cons[i,t] = model.addConstr(
            X[i,t] + sD[i,t-1]
            <=
            quicksum(dc_ub[ct] * y[i,ct,t] for ct in  CT)
        )
    
model.update()

In [7]:
model.setObjective(
    quicksum(transport[i,t] * X[i,t] for i in DC for t in Period) +
    quicksum(delivery[i,j] * x[i,j,t] for i,j in Route for t in Period) +
    quicksum(dc_run[i,ct] * y[i,ct,t] for i in DC for ct in CT for t in Period) +
    quicksum(dc_stock[i] * sD[i,t] for i in DC for t in Period)
    ,GRB.MINIMIZE
)

model.update()

In [8]:
model.optimize()

Optimize a model with 48 rows, 50 columns and 152 nonzeros
Variable types: 34 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [1e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 2020.0000000
Presolve removed 18 rows and 18 columns
Presolve time: 0.00s
Presolved: 30 rows, 32 columns, 94 nonzeros
Variable types: 16 continuous, 16 integer (16 binary)

Root relaxation: objective 1.150000e+03, 19 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1150.00000    0    8 2020.00000 1150.00000  43.1%     -    0s
H    0     0                    1200.0000000 1150.00000  4.17%     -    0s
     0     0 1170.00000    0    5 1200.00000 1170.00000  2.50%     -    0s

Cutting planes:
  Gomory: 4
  MIR: 1

Explored 1 nodes (28 simplex iter

In [9]:
for v in model.getVars():
    if v.X > 0:
        print(v.VarName, v.X)

X[1,1] 40.0
X[2,1] 40.0
x[1,2,1] 10.0
x[1,2,2] 10.0
x[1,1,1] 10.0
x[1,1,2] 10.0
x[2,1,3] 10.0
x[2,1,4] 10.0
x[2,2,3] 10.0
x[2,2,4] 10.0
y[1,1,1] 1.0
y[1,1,2] 1.0
y[2,1,1] 1.0
y[2,1,2] 1.0
y[2,1,3] 1.0
y[2,1,4] 1.0
sD[1,1] 20.0
sD[2,1] 40.0
sD[2,2] 40.0
sD[2,3] 20.0
